In [66]:
import os
from collections import namedtuple

Note: pylint can be run on whole directories!

In [58]:
test_target = './spellbook/familiars/inspector_gadget/gadget.py'

In [59]:
def load_text(path):
    with open(os.path.join(path), 'r') as temp_file:
        text = temp_file.read()
    return text

In [60]:
def lint_load_cleanup(linting_target_path, temp_file_path=None):
    if temp_file_path is None:
        raw_output_file = os.path.join('./temp_pylint_output.txt')
    else:
        raw_output_file = os.path.join(temp_file_path)
    linting_target = os.path.join(linting_target_path)
    os.system(f'python -m pylint {linting_target_path} > {raw_output_file}')
    text = load_text(raw_output_file)
    os.system(f'rm {raw_output_file}')
    return text

In [61]:
def raw_text_to_lines(text):
    ''''''
    output = tuple(line.strip() for line in text.strip().split('\n') if line)
    return output

In [62]:
def partition_lines(lines):
    header = lines[0]
    body = lines[1:-2]
    score = lines[-1]
    return header, body, score

In [63]:
def extract_score_line_tuple(preprocessed_lines):
    ''''''
    expected_line = preprocessed_lines[-1]
    if (len(expected_line) > 9) and (expected_line[0:9] == "Your code"):
        return tuple(expected_line.split())
    else:
        for line in preprocessed_lines:
            if (len(line) > 9) and (line[0:9] == "Your code"):
                return tuple(line.split())
    raise RuntimeError("no score line was found!")

In [69]:
Score = namedtuple('Score', ('current', 'previous', 'delta'))

def parse_score(score_line):
    words = score_line.split()
    current = words[6]
    if len(words) == 7:
        previous = None
        delta = None
    else:
        previous = words[9][:-1]
        delta = words[10][:-1]
    score_holder = Score(current, previous, delta)
    return score_holder

In [75]:
def numeric_score(score):
    assert isinstance(score, Score)
    num_score = Score(
        float(score.current.split('/')[0]),
        float(score.previous.split('/')[0]),
        float(score.delta)
    )
    return num_score

In [77]:
text = lint_load_cleanup(test_target)
lines = raw_text_to_lines(text)
header_line, body_lines, score_line = partition_lines(lines)
score = parse_score(score_line)
num_score = numeric_score(score)

In [78]:
print(score)
print(num_score)

Score(current='3.52/10', previous='3.52/10', delta='+0.00')
Score(current=3.52, previous=3.52, delta=0.0)


In [79]:
from itertools import takewhile

In [101]:
PylintItem = namedtuple('PylintItem', ('line_number', 'error_code', 'concise', 'message', 'unknown'))

def body_to_row_tuples(body_lines):
    ''''''
    colon_split = [line.split(':') for line in body_lines]
    clean_lines = [[entry.strip() for entry in line[1:]] for line in colon_split]
    output = []
    for line in clean_lines:
        if len(line) != 4:
            raise NotImplementedError('No handling for this line format!')
        
        line_number = line[0]
        unknown = line[1]
        error_code = line[2]
        message = ''.join([char for char in takewhile(lambda x: x != '(', line[3])][:-1])
        concise = line[3].split('(')[-1][:-1]
        
        final_line = PylintItem(
            line_number,
            error_code,
            concise,
            message,
            unknown
        )
        output.append(final_line)
    return output

In [92]:
import pandas as pd

In [102]:
def rows_to_dataframe(pylint_items):
    columns = {
        'line_number': [],
        'error_code': [],
        'concise': [],
        'message': [],
        'unknown': [],
    }
    
    for item in pylint_items:
        columns['line_number'].append(item.line_number)
        columns['error_code'].append(item.error_code)
        columns['concise'].append(item.concise)
        columns['message'].append(item.message)
        columns['unknown'].append(item.unknown)
    df = pd.DataFrame(data=columns)
    return df
        
        

In [103]:
rows = body_to_row_tuples(body_lines)
rows

[PylintItem(line_number='1', error_code='C0111', concise='missing-docstring', message='Missing module docstring', unknown='0'),
 PylintItem(line_number='13', error_code='C0103', concise='invalid-name', message='Constant name "typedefs" doesn\'t conform to UPPER_CASE naming style', unknown='0'),
 PylintItem(line_number='19', error_code='C0111', concise='missing-docstring', message='Missing function docstring', unknown='0'),
 PylintItem(line_number='19', error_code='W0622', concise='redefined-builtin', message="Redefining built-in 'object'", unknown='12'),
 PylintItem(line_number='23', error_code='C0103', concise='invalid-name', message='Constant name "common_attrs" doesn\'t conform to UPPER_CASE naming style', unknown='0'),
 PylintItem(line_number='25', error_code='C0103', concise='invalid-name', message='Constant name "all_attrs" doesn\'t conform to UPPER_CASE naming style', unknown='0'),
 PylintItem(line_number='61', error_code='C0111', concise='missing-docstring', message='Missing cl

In [104]:
df = rows_to_dataframe(rows)
df

,line_number,error_code,concise,message,unknown
0,1,C0111,missing-docstring,Missing module docstring,0
1,13,C0103,invalid-name,"Constant name ""typedefs"" doesn't conform to UP...",0
2,19,C0111,missing-docstring,Missing function docstring,0
3,19,W0622,redefined-builtin,Redefining built-in 'object',12
4,23,C0103,invalid-name,"Constant name ""common_attrs"" doesn't conform t...",0
5,25,C0103,invalid-name,"Constant name ""all_attrs"" doesn't conform to U...",0
6,61,C0111,missing-docstring,Missing class docstring,0
7,66,C0111,missing-docstring,Missing method docstring,4
8,74,C0111,missing-docstring,Missing method docstring,4
9,79,C0111,missing-docstring,Missing method docstring,4


[PylintItem(line='1', code='C0111', concise='missing-docstring', message='Missing module docstring', unknown='0'),
 PylintItem(line='13', code='C0103', concise='invalid-name', message='Constant name "typedefs" doesn\'t conform to UPPER_CASE naming style', unknown='0'),
 PylintItem(line='19', code='C0111', concise='missing-docstring', message='Missing function docstring', unknown='0'),
 PylintItem(line='19', code='W0622', concise='redefined-builtin', message="Redefining built-in 'object'", unknown='12'),
 PylintItem(line='23', code='C0103', concise='invalid-name', message='Constant name "common_attrs" doesn\'t conform to UPPER_CASE naming style', unknown='0'),
 PylintItem(line='25', code='C0103', concise='invalid-name', message='Constant name "all_attrs" doesn\'t conform to UPPER_CASE naming style', unknown='0'),
 PylintItem(line='61', code='C0111', concise='missing-docstring', message='Missing class docstring', unknown='0'),
 PylintItem(line='66', code='C0111', concise='missing-docstri

In [85]:
for entry in takewhile(lambda x: x != '(', [1, 2, 3, 4, 5, '(', 8, 9]):
    print(entry)

1
2
3
4
5


In [80]:
help(takewhile)

Help on class takewhile in module itertools:

class takewhile(builtins.object)
 |  takewhile(predicate, iterable) --> takewhile object
 |  
 |  Return successive entries from an iterable as long as the 
 |  predicate evaluates to true for each entry.
 |  
 |  Methods defined here:
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.
 |  
 |  __next__(self, /)
 |      Implement next(self).
 |  
 |  __reduce__(...)
 |      Return state information for pickling.
 |  
 |  __setstate__(...)
 |      Set state information for unpickling.



In [55]:
text = lint_load_cleanup(test_target)

In [57]:
lines

('************* Module inspector_gadget.gadget',
 'spellbook\\familiars\\inspector_gadget\\gadget.py:1:0: C0111: Missing module docstring (missing-docstring)',
 'spellbook\\familiars\\inspector_gadget\\gadget.py:13:0: C0103: Constant name "typedefs" doesn\'t conform to UPPER_CASE naming style (invalid-name)',
 'spellbook\\familiars\\inspector_gadget\\gadget.py:19:0: C0111: Missing function docstring (missing-docstring)',
 "spellbook\\familiars\\inspector_gadget\\gadget.py:19:12: W0622: Redefining built-in 'object' (redefined-builtin)",
 'spellbook\\familiars\\inspector_gadget\\gadget.py:23:0: C0103: Constant name "common_attrs" doesn\'t conform to UPPER_CASE naming style (invalid-name)',
 'spellbook\\familiars\\inspector_gadget\\gadget.py:25:0: C0103: Constant name "all_attrs" doesn\'t conform to UPPER_CASE naming style (invalid-name)',
 'spellbook\\familiars\\inspector_gadget\\gadget.py:61:0: C0111: Missing class docstring (missing-docstring)',
 'spellbook\\familiars\\inspector_gadget

In [56]:
lines = raw_text_to_lines(text)

In [53]:
partition_lines(lines)

('************* Module inspector_gadget.inspections',
 ('spellbook\\familiars\\inspector_gadget\\inspections.py:1:0: C0111: Missing module docstring (missing-docstring)',
  'spellbook\\familiars\\inspector_gadget\\inspections.py:13:0: C0103: Constant name "typedefs" doesn\'t conform to UPPER_CASE naming style (invalid-name)',
  'spellbook\\familiars\\inspector_gadget\\inspections.py:19:0: C0111: Missing function docstring (missing-docstring)',
  "spellbook\\familiars\\inspector_gadget\\inspections.py:19:12: W0622: Redefining built-in 'object' (redefined-builtin)",
  'spellbook\\familiars\\inspector_gadget\\inspections.py:23:0: C0103: Constant name "common_attrs" doesn\'t conform to UPPER_CASE naming style (invalid-name)',
  'spellbook\\familiars\\inspector_gadget\\inspections.py:25:0: C0103: Constant name "all_attrs" doesn\'t conform to UPPER_CASE naming style (invalid-name)',
  'spellbook\\familiars\\inspector_gadget\\inspections.py:72:0: C0111: Missing class docstring (missing-docstr

In [56]:
score_string = extract_score_string(lines)

In [48]:
len(score_string.split())

7

In [46]:
metric_strings = metrics_from_score_string(score_string)

IndexError: list index out of range

In [45]:
score_string, metric_strings

('Your code has been rated at 1.67/10', ('8.41/10', '8.41/10', '+0.00'))

In [38]:
lines

('************* Module inspector_gadget.inspections',
 'spellbook\\familiars\\inspector_gadget\\inspections.py:1:0: C0111: Missing module docstring (missing-docstring)',
 'spellbook\\familiars\\inspector_gadget\\inspections.py:13:0: C0103: Constant name "typedefs" doesn\'t conform to UPPER_CASE naming style (invalid-name)',
 'spellbook\\familiars\\inspector_gadget\\inspections.py:19:0: C0111: Missing function docstring (missing-docstring)',
 "spellbook\\familiars\\inspector_gadget\\inspections.py:19:12: W0622: Redefining built-in 'object' (redefined-builtin)",
 'spellbook\\familiars\\inspector_gadget\\inspections.py:23:0: C0103: Constant name "common_attrs" doesn\'t conform to UPPER_CASE naming style (invalid-name)',
 'spellbook\\familiars\\inspector_gadget\\inspections.py:25:0: C0103: Constant name "all_attrs" doesn\'t conform to UPPER_CASE naming style (invalid-name)',
 'spellbook\\familiars\\inspector_gadget\\inspections.py:72:0: C0111: Missing class docstring (missing-docstring)',


In [42]:
score_string(lines)

TypeError: 'str' object is not callable

In [41]:
body_to_row_tuples(lines)

['spellbook\\familiars\\inspector_gadget\\inspections.py', '1', '0', 'C0111', 'Missing module docstring (missing-docstring)']


[]

In [64]:
def full_pipe(target_path):
    text = lint_load_cleanup(target_path)
    lines = raw_text_to_lines(text)
    score_string = extract_score_string(lines)
    current, previous, delta = metrics_from_score_string(score_string)
    return lines

In [65]:
full_pipe(test_target)

Your code has been rated at 8.41/10 (previous run: 8.41/10, +0.00)


('************* Module core.parsing.flake8_parse',
 'core\\parsing\\flake8_parse.py:1:0: C0111: Missing module docstring (missing-docstring)',
 'core\\parsing\\flake8_parse.py:9:4: R1705: Unnecessary "else" after "return" (no-else-return)',
 'core\\parsing\\flake8_parse.py:25:4: R1705: Unnecessary "else" after "return" (no-else-return)',
 'core\\parsing\\flake8_parse.py:62:0: C0111: Missing function docstring (missing-docstring)',
 'core\\parsing\\flake8_parse.py:68:4: R1705: Unnecessary "else" after "return" (no-else-return)',
 'core\\parsing\\flake8_parse.py:3:0: C0411: standard import "from typing import List, Optional" should be placed before "import pandas as pd" (wrong-import-order)',
 'core\\parsing\\flake8_parse.py:4:0: C0411: standard import "from collections import OrderedDict" should be placed before "import pandas as pd" (wrong-import-order)',
 '------------------------------------------------------------------',
 'Your code has been rated at 8.41/10 (previous run: 8.41/10,

In [53]:
lint_load_cleanup(test_target)

'************* Module core.parsing.flake8_parse\ncore\\parsing\\flake8_parse.py:1:0: C0111: Missing module docstring (missing-docstring)\ncore\\parsing\\flake8_parse.py:9:4: R1705: Unnecessary "else" after "return" (no-else-return)\ncore\\parsing\\flake8_parse.py:25:4: R1705: Unnecessary "else" after "return" (no-else-return)\ncore\\parsing\\flake8_parse.py:62:0: C0111: Missing function docstring (missing-docstring)\ncore\\parsing\\flake8_parse.py:68:4: R1705: Unnecessary "else" after "return" (no-else-return)\ncore\\parsing\\flake8_parse.py:3:0: C0411: standard import "from typing import List, Optional" should be placed before "import pandas as pd" (wrong-import-order)\ncore\\parsing\\flake8_parse.py:4:0: C0411: standard import "from collections import OrderedDict" should be placed before "import pandas as pd" (wrong-import-order)\n\n-----------------------------------\n\nYour code has been rated at 8.41/10\n\n\n\n'

In [21]:
%%bash
cat 'temp_pylint_output.txt'

************* Module core.parsing.pylint_parse
core\parsing\pylint_parse.py:1:0: C0111: Missing module docstring (missing-docstring)
core\parsing\pylint_parse.py:27:15: C1801: Do not use `len(SEQUENCE)` to determine if a sequence is empty (len-as-condition)
core\parsing\pylint_parse.py:30:15: C1801: Do not use `len(SEQUENCE)` to determine if a sequence is empty (len-as-condition)
core\parsing\pylint_parse.py:126:0: C0111: Missing function docstring (missing-docstring)
core\parsing\pylint_parse.py:2:0: C0411: standard import "from typing import List, Tuple" should be placed before "from IPython.utils.capture import CapturedIO" (wrong-import-order)
core\parsing\pylint_parse.py:3:0: C0411: standard import "from collections import OrderedDict" should be placed before "from IPython.utils.capture import CapturedIO" (wrong-import-order)

-----------------------------------
Your code has been rated at 9.26/10



In [37]:
pylint_pre_process(load_text(target))

['************* Module core.parsing.pylint_parse',
 'core\\parsing\\pylint_parse.py:1:0: C0111: Missing module docstring (missing-docstring)',
 'core\\parsing\\pylint_parse.py:27:15: C1801: Do not use `len(SEQUENCE)` to determine if a sequence is empty (len-as-condition)',
 'core\\parsing\\pylint_parse.py:30:15: C1801: Do not use `len(SEQUENCE)` to determine if a sequence is empty (len-as-condition)',
 'core\\parsing\\pylint_parse.py:126:0: C0111: Missing function docstring (missing-docstring)',
 'core\\parsing\\pylint_parse.py:2:0: C0411: standard import "from typing import List, Tuple" should be placed before "from IPython.utils.capture import CapturedIO" (wrong-import-order)',
 'core\\parsing\\pylint_parse.py:3:0: C0411: standard import "from collections import OrderedDict" should be placed before "from IPython.utils.capture import CapturedIO" (wrong-import-order)',
 '',
 '-----------------------------------',
 '',
 'Your code has been rated at 9.26/10']